In [5]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, fcluster
df=pd.read_csv('data.csv',encoding='unicode_escape')

In [6]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


1) Preprocess the data. How are you handling nulls? What process(es) are you using to encode and normalize the data?

In [7]:
df.dropna(inplace=True)
#I handled the nulls by removing the rows with missing data
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M')
df["Description"] = df["Description"].str.lower()


df = df[df['Quantity'] > 0]

df.insert(loc=2, column='year_month', value=df['InvoiceDate'].map(lambda x: 100*x.year + x.month))

df.insert(loc=3, column='month', value=df.InvoiceDate.dt.month)


df.insert(loc=4, column='day', value=(df.InvoiceDate.dt.dayofweek)+1)

df.insert(loc=5, column='hour', value=df.InvoiceDate.dt.hour)

df["amount_spent"] = df["UnitPrice"]*df["Quantity"]


#Here I made the data easier to read by changing formatting and also adding a column to help understand the 
#data better.



/var/folders/68/jgh30zmn54z999vhrzlb3zbc0000gn/T/ipykernel_5104/1813979738.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["amount_spent"] = df["UnitPrice"]*df["Quantity"]


Perform k-means on this dataset for customer segments. Customer segments help determine what types of people buy your product, which allows you to target more people like your usual customers. Should you look at all the data, or which subset of data should you use? What is the ideal number of clusters? Which approach did you use to find the ideal number of clusters and why?

In [8]:
intertia = []
customer_features = ['amount_spent', 'Quantity', 'InvoiceNo']
X = df[customer_features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_scaled)
df['clusters'] = kmeans.labels_

def elbow_method(X, k):
  kmeans = KMeans(n_clusters=k, random_state=0)
  kmeans.fit(X)
  return kmeans.inertia_

print(kmeans.cluster_centers_)



[[-1.99714330e-03 -2.16154475e-03 -3.22085130e-07]
 [ 3.97353629e+02  4.30063105e+02  6.40823794e-02]]


/var/folders/68/jgh30zmn54z999vhrzlb3zbc0000gn/T/ipykernel_5104/322391351.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clusters'] = kmeans.labels_


You should not use the entire dataset, but the sections that show frequency and amount spent. This way we know how much each customer bought and how much they spent, which allowed us to figure out what is more popular. 



 Perform hierarchical clustering for customer segments. What is the ideal number of clusters? Which approach did you use to find the ideal number of clusters and why?

In [9]:
customer_features = ['amount_spent', 'Quantity', 'InvoiceNo']
X = df[customer_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


linkage_method = 'ward'
distance_matrix = linkage(X_scaled, method=linkage_method)

threshold = 5
cluster_labels = fcluster(distance_matrix, threshold, method=linkage_method)
df["cluster_label"] = cluster_labels

number_of_clusters = len(set(cluster_labels))
print(number_of_clusters)



 Compare the results of 2 and 3. Which approach do you think is best? Why?